In [6]:
import torch
torch.__version__
import torch.nn as nn  # 这行代码就是告诉 Python：以后我写 nn，就是指 torch.nn

In [11]:
class LogisticRegression(nn.Module):
    def __init__(self):
        super().__init__()
        # 只有一层：直接从输入到输出
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [12]:
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 第一层：输入 -> 隐藏层 (比如 128 个神经元)
        self.layer1 = nn.Linear(2, 128)
        self.relu = nn.ReLU() # 激活函数 (换成 ReLU 效果更好)

        # 第二层：隐藏层 -> 输出
        self.layer2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)   # 增加非线性能力
        x = self.layer2(x)
        return self.sigmoid(x)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

# --- 假设你上面已经定义好了 SimpleNeuralNet 类 ---
# class SimpleNeuralNet(nn.Module): ... (略)

# 1. 准备数据 (这里用一个经典的“异或 XOR”问题)
# 逻辑：两数相同为0，不同为1。这是逻辑回归解决不了，但神经网络能解决的问题。
X = torch.tensor([[0.0, 0.0],
                  [0.0, 1.0],
                  [1.0, 0.0],
                  [1.0, 1.0]])  # 输入 (4个样本, 每个2维)

y = torch.tensor([[0.0],
                  [1.0],
                  [1.0],
                  [0.0]])       # 标签 (4个样本, 每个1维)

# 2. 把模型和数据搬到你的 RTX 4080 上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"当前使用的设备: {device}")

model = SimpleNeuralNet().to(device)  # 实例化模型并丢进显卡
X = X.to(device)
y = y.to(device)

# 3. 定义“裁判”和“教练”
# 裁判：损失函数 (BCELoss 用于二分类)
criterion = nn.BCELoss()
# 教练：优化器 (Adam 通常比 SGD 收敛更快，适合新手)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 4. 开始训练 (Training Loop)
print("开始训练...")
for epoch in range(2000):  # 训练 2000 次
    # --- 前向传播 (Forward) ---
    y_pred = model(X)      # 1. 喂数据，出结果
    loss = criterion(y_pred, y) # 2. 算差距 (Loss)

    # --- 反向传播 (Backward) ---
    optimizer.zero_grad()  # 3. 梯度归零 (防止上次的梯度残留)
    loss.backward()        # 4. 计算梯度 (反向找锅)
    optimizer.step()       # 5. 更新权重 (根据梯度调整参数)

    # 每 200 次打印一下进度
    if (epoch + 1) % 200 == 0:
        print(f'Epoch [{epoch+1}/2000], Loss: {loss.item():.4f}')

# 5. 测试使用 (Inference)
print("\n训练结束，来测试一下预测结果：")
with torch.no_grad(): # 测试时不需要算梯度，省显存
    test_input = torch.tensor([[0.0, 1.0]]).to(device) # 测试数据：0 和 1
    prediction = model(test_input)
    print(f"输入 [0, 1] -> 预测值: {prediction.item():.4f} (应该是接近 1)")

    test_input2 = torch.tensor([[1.0, 1.0]]).to(device) # 测试数据：1 和 1
    prediction2 = model(test_input2)
    print(f"输入 [1, 1] -> 预测值: {prediction2.item():.4f} (应该是接近 0)")

当前使用的设备: cuda
开始训练...
Epoch [200/2000], Loss: 0.0013
Epoch [400/2000], Loss: 0.0004
Epoch [600/2000], Loss: 0.0002
Epoch [800/2000], Loss: 0.0001
Epoch [1000/2000], Loss: 0.0001
Epoch [1200/2000], Loss: 0.0000
Epoch [1400/2000], Loss: 0.0000
Epoch [1600/2000], Loss: 0.0000
Epoch [1800/2000], Loss: 0.0000
Epoch [2000/2000], Loss: 0.0000

训练结束，来测试一下预测结果：
输入 [0, 1] -> 预测值: 1.0000 (应该是接近 1)
输入 [1, 1] -> 预测值: 0.0000 (应该是接近 0)
